<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [44]:
!pip3 install folium
!pip3 install wget

In [45]:
import folium
import wget
import pandas as pd

In [46]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [47]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [48]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [49]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [50]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='blue', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map



An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [51]:
###  RE-visit our 'launch sites' df

launch_sites_df.head()

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [52]:
# do a 're-name'
launch_sites_df.rename(columns = {'Launch Site' : 'LaunchSite'}, inplace = True)

In [53]:
# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# For each launch site, add a Circle object based on its 
# coordinate (Lat, Long) values. In addition, add Launch site 
# name as a popup label

for lat, long, name  in zip(launch_sites_df.Lat, launch_sites_df.Long, launch_sites_df.LaunchSite):
    # add 'Circle'
    site_map.add_child(folium.Circle([lat,long],radius=250, color='#d35400', fill=True).add_child(folium.Popup(name))
                      )
    # add 'Marker'
    site_map.add_child(folium.map.Marker([lat,long],
                       icon=DivIcon(icon_size=(20,20),
                       icon_anchor=(0,0),
                       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % name,))
                      )
site_map


In [54]:
##  GET a 'global map' for the CAPSTONE



The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png" />
</center>


In [55]:
##  GET a 'global map' for the CAPSTONE

# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)

# For each launch site, add a Circle object based on its 
# coordinate (Lat, Long) values. In addition, add Launch site 
# name as a popup label

for lat, long, name  in zip(launch_sites_df.Lat, launch_sites_df.Long, launch_sites_df.LaunchSite):
    # add 'Circle'
    site_map.add_child(folium.Circle([lat,long],radius=250, color='#d35400', fill=True).add_child(folium.Popup(name))
                      )
    # add 'Marker'
    site_map.add_child(folium.map.Marker([lat,long],
                       icon=DivIcon(icon_size=(20,20),
                       icon_anchor=(0,0),
                       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % name,))
                      )
site_map

Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

Also please try to explain your findings.

*   Are all launch sites in proximity to the Equator line? Explanation: No. All launch sites are well north of the Equator by at least 28 degrees latitude.
*   Are all launch sites in very close proximity to the coast? Explanation: Yes. VAFB SLC 4E is located on the US west coast on the Pacific Ocean. CCAFS LC-40, CCAFS SLC-40, and KSC LC-39A are all located on the Alantic coast of the US in the Merritt Island area of Florida.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [56]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`







Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [57]:
marker_cluster = MarkerCluster()

*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [58]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red


In [59]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [60]:
spacex_df.head(10)

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red
5,CCAFS LC-40,28.562302,-80.577356,0,red
6,CCAFS LC-40,28.562302,-80.577356,0,red
7,CCAFS LC-40,28.562302,-80.577356,0,red
8,CCAFS LC-40,28.562302,-80.577356,0,red
9,CCAFS LC-40,28.562302,-80.577356,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [61]:

# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker = folium.Marker([record['Lat'], record['Long']],
                          icon = folium.Icon(color = 'white', icon_color = record['marker_color']))

    marker_cluster.add_child(marker)

site_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png" />
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png" />
</center>




From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [62]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [63]:
#


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.



You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [64]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance


*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


Create a marker with distance to a closest coastline.

TO the nearest COASTLINE POINT
Draw a line between the marker to the launch site .............. we do 'to Atlanic Coastline'

In [65]:
# find coordinate of the closet coastline

fl_coastline_lat = 28.56224
fl_coastline_lon = -80.56787

fl_launch_site_lat = 28.562302
fl_launch_site_lon = -80.577356
fl_distance_coastline = calculate_distance(fl_launch_site_lat, fl_launch_site_lon, fl_coastline_lat, fl_coastline_lon)

print('The distance from launch site CCAFS LC-40 to the coastline is :', round(fl_distance_coastline, 3), 'km')


The distance from launch site CCAFS LC-40 to the coastline is : 0.927 km


In [66]:
## for Florida

In [67]:
##  Going on......
fl_coastline_coord = fl_coastline_lat, fl_coastline_lon
fl_launch_site_coord = fl_launch_site_lat, fl_launch_site_lon

*TODO:* After obtained its coordinate, create a `folium.Marker` to show the distance


In [68]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 

fl_distance_marker = folium.Marker(
    fl_coastline_coord,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} km".format(fl_distance_coastline),
        )
    )

In [69]:
#

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [70]:
##  FOR FLORIDA

In [71]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
#coastline_coord = coastline_lat, coastline_lon 
fl_coordinates = [fl_launch_site_coord, fl_coastline_coord]
lines = folium.PolyLine(locations = fl_coordinates, weight = 1)
#site_map.add_child(lines)

In [73]:
##  We 'add the following' ::
# Create a blue circle at the Coastlines's coordinate with a popup label showing its name
circle = folium.Circle(fl_coastline_coord, radius=75, color = 'light blue', fill = True).add_child(folium.Popup('Coastline Point'))

    
site_map.add_child(lines)
site_map.add_child(circle)
site_map.add_child(fl_distance_marker)

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png" />
</center>


In [75]:
## for California
# find coordinate of the closet coastline

vafb_coastline_lat = 34.63619
vafb_coastline_lon = -120.62488

vafb_launch_site_lat = 34.632834
vafb_launch_site_lon = -120.61074
vafb_distance_coastline = calculate_distance(vafb_launch_site_lat, vafb_launch_site_lon, vafb_coastline_lat, vafb_coastline_lon)

print('The distance from launch site VAFB SLC-4E to the coastline is :', round(vafb_distance_coastline, 3), 'km')


The distance from launch site VAFB SLC-4E to the coastline is : 1.347 km


In [76]:
##    VAFB
##  Going on......
vafb_coastline_coord = vafb_coastline_lat, vafb_coastline_lon
vafb_launch_site_coord = vafb_launch_site_lat, vafb_launch_site_lon

In [77]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 

vafb_distance_marker = folium.Marker(
    vafb_coastline_coord,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} km".format(vafb_distance_coastline),
        )
    )

In [78]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
#coastline_coord = coastline_lat, coastline_lon 
vafb_coordinates = [vafb_launch_site_coord, vafb_coastline_coord]
lines = folium.PolyLine(locations = vafb_coordinates, weight = 1)
#site_map.add_child(lines)

In [79]:
##  We 'add the following' ::
# Create a blue circle at the Coastlines's coordinate with a popup label showing its name
circle = folium.Circle(vafb_coastline_coord, radius=75, color = 'light blue', fill = True).add_child(folium.Popup('Coastline Point'))

    
site_map.add_child(lines)
site_map.add_child(circle)
site_map.add_child(vafb_distance_marker)

*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png" />
</center>


Create a marker with distance to a closest city, railway, highway, etc.

TO the nearest Railway
Draw a line between the marker to the launch site .............. we do 'to NASA Railroad'¶


In [80]:
## TO the nearest RAILWAY

# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site --------- we choose to do the railway (RR) distance
##                                                                      from the launch_site 1st...
##   Calculate 'distance'
RR_lat = 28.57216
RR_lon = -80.5853

launch_site_lat = 28.562302
launch_site_lon = -80.577356
distance_RR = calculate_distance(launch_site_lat, launch_site_lon, RR_lat, RR_lon)

print('The distance from launch site to the NASA Railroad is :', round(distance_RR, 3), 'km')


The distance from launch site to the NASA Railroad is : 1.343 km


In [81]:
## 
RR_coord = RR_lat, RR_lon
## the 'didstance marker'
distance_marker = folium.Marker(
    RR_coord,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(5,5),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_RR),
        )
    )
## the...'PolyLine' 
coordinates = [[launch_site_lat, launch_site_lon], RR_coord]
lines=folium.PolyLine(locations=coordinates, weight=1)
## 'circle and pop-up'
circle = folium.Circle(RR_coord, radius=75, color='#d35400', fill=True).add_child(folium.Popup('Railway Point'))

## put to map    
site_map.add_child(lines)
site_map.add_child(circle)
site_map.add_child(distance_marker)


In [82]:
#

A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png" />
</center>


Create a marker with distance to a closest city, railway, highway, etc.

TO the nearest HIGHWAY
Draw a line between the marker to the launch site .............. we do 'to Sam Phillips Pkwy'


In [83]:
## TO the nearest Highway
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site .............. we do 'to Samuel Phillips Parkway'
##                                                                      
##   Calculate 'distance'
hwy_lat = 28.562302
hwy_lon = -80.57068

launch_site_lat = 28.562302
launch_site_lon = -80.577356
distance_hwy = calculate_distance(launch_site_lat, launch_site_lon, hwy_lat, hwy_lon)

print('The distance from launch site to the Sam Phillips Parkway is :', round(distance_hwy, 3), 'km')

The distance from launch site to the Sam Phillips Parkway is : 0.652 km


In [84]:
## 
hwy_coord = hwy_lat, hwy_lon
## the 'didstance marker'
distance_marker = folium.Marker(
    hwy_coord,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(5,5),
        html='<div style="font-size: 12; color:#53bbb4;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_hwy),
        )
    )
## the...'PolyLine' 
coordinates = [[launch_site_lat, launch_site_lon], hwy_coord]
lines=folium.PolyLine(locations=coordinates, weight=1)
## 'circle and pop-up'
circle = folium.Circle(hwy_coord, radius=75, color='#53bbb4', fill=True).add_child(folium.Popup('Highway Point'))

## put to map    
site_map.add_child(lines)
site_map.add_child(circle)
site_map.add_child(distance_marker)

In [30]:
#

A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png" />
</center>


Create a marker with distance to a closest city, railway, highway, etc.

TO the nearest CITY
Draw a line between the marker to the launch site .............. we do 'to city of Cape Canaveral'

In [85]:
## TO the nearest CITY
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site .............. we do 'to city of Cape Canaveral'
##                                                                      
##   Calculate 'distance'
city_lat = 28.40771
city_lon = -80.59193

launch_site_lat = 28.562302
launch_site_lon = -80.577356
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

print('The distance from launch site to the City of Cape Canaveral is :', round(distance_city, 3), 'km')

The distance from launch site to the City of Cape Canaveral is : 17.254 km


In [86]:
## see the 'distance' statement immediately above

In [87]:
## 
city_coord = city_lat, city_lon
## the 'didstance marker'
distance_marker = folium.Marker(
    city_coord,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(5,5),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
        )
    )
## the...'PolyLine' 
coordinates = [[launch_site_lat, launch_site_lon], city_coord]
lines=folium.PolyLine(locations=coordinates, weight=1)
## 'circle and pop-up'
circle = folium.Circle(city_coord, radius=75, color='#d35400', fill=True).add_child(folium.Popup('City of Cape Canaveral Point'))

## put to map    
site_map.add_child(lines)
site_map.add_child(circle)
site_map.add_child(distance_marker)

In [34]:
#

Create a marker with distance to a closest city, railway, highway, etc.

WE ALSO CHECK THE DISTANCE FROM KSC LC-39A TO TITUSVILLE....

TO the nearest CITY
Draw a line between the marker to the launch site .............. we do ' KSC LC-39A to Titusville'

In [35]:
#

In [36]:
## TO the nearest CITY
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site .............. we do 'KSC LC-39A to city of Titusville'
##                                                                      
##   Calculate 'distance'
city_lat = 28.57331
city_lon = -80.79741

launch_site_lat = 28.57331
launch_site_lon = -80.64694
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

print('The distance from launch site to the City of Titusville is :', round(distance_city, 3), 'km')

The distance from launch site to the City of Titusville is : 14.698 km


In [37]:
## see the 'distance' statement immediately above

In [38]:
## 
city_coord = city_lat, city_lon
## the 'didstance marker'
distance_marker = folium.Marker(
    city_coord,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(5,5),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
        )
    )
## the...'PolyLine' 
coordinates = [[launch_site_lat, launch_site_lon], city_coord]
lines=folium.PolyLine(locations=coordinates, weight=1)
## 'circle and pop-up'
circle = folium.Circle(city_coord, radius=75, color='#d35400', fill=True).add_child(folium.Popup('City Titusville Point'))

## put to map    
site_map.add_child(lines)
site_map.add_child(circle)
site_map.add_child(distance_marker)

In [39]:
#

After you plot distance lines to the proximities, you can answer the following questions easily:

Also please try to explain your findings.

*   Are launch sites in close proximity to railways? Explanation : Yes. Launch Site CCAFS LC-40 is just 1.34 km from the NASA Railroad termination. Launch Site CCAFS SLC-40 is ALSO just about 1.34 km from the NASA Railroad termination.

*   Are launch sites in close proximity to highways? Explanation : Yes. Launch Site CCAFS LC-40 is just 0.625 km from the Sam Pillips Parkway. Launch Site CCAFS SLC-40 appears to be less than 0.625 km from the Sam Pillips Parkway.
*   
*   Are launch sites in close proximity to coastline? Explanation : Yes. Launch Site CCAFS LC-40 is just 0.925 km from the Atlantic Coast. Launch Site CCAFS SLC-40 appears to be less than 0.925 km from the Atlantic Coast.

*   Do launch sites keep certain distance away from cities? Explanation : Yes. Launch Site CCAFS LC-40 is 17.25 km from the nearest city of Cape Canaveral. Launch Site CCAFS SLC-40 appears to be more than 17.25 km from the city of Cape Canaveral. Secondarily, Launch Site KSC LC-39A is 14.70 km from the nearest city of Titusville. 





In [40]:
# LAB Complete

# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01)


### Other Contributors


Joseph Santarcangelo


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description          |
| ----------------- | ------- | ---------- | --------------------------- |
| 2021-05-26        | 1.0     | Yan        | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
